In [2]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [30]:
res = np.array([[706, 297, 569, 856, 721, 293, 503, 822, 617, 281, 478, 810],
                [906, 382, 732, 930, 892, 350, 586, 907, 735, 332, 551, 857],
                [842, 328, 695, 896, 821, 319, 564, 861, 674, 301, 515, 854],
                [869, 341, 724, 921, 848, 334, 587, 890, 702, 307, 538, 869]])

In [31]:
rrk = (res[1] - res[0]) / res[0]

In [32]:
np.average(rrk[:4]), np.average(rrk[4:8]), np.average(rrk[8:])

(0.23559937253131952, 0.1750315279831428, 0.1458717481549427)

In [20]:
rrk[np.array([0,4,8])], rrk[1,5,9], rrk[2,6,10], rrk[3,7,11]

IndexError: too many indices for array

In [59]:
np.average(rrk[np.array([0,4,8])]), np.average(rrk[np.array([1,5,9])]), np.average(rrk[np.array([2,6,10])]), np.average(rrk[np.array([3,7,11])])

(0.24338090279766775,
 0.22074306575458402,
 0.20139903081626667,
 0.07926670988145214)

In [ ]:
res = np.array([[696, 297, 569, 836, 721, 293, 503, 822, 617, 281, 478, 876],
                [906, 382, 732, 930, 892, 350, 586, 907, 735, 332, 551, 918]])

In [2]:
data_dir = '/datadrive/data_cs/'
batch_size = 256
batch_num  = 128
epoch_num  = 200
samp_num   = 7

device = torch.device("cuda:3")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t < 2015}
valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [4]:
def pf_sample(seed, papers, pairs, t_range, batch_size, test = False):
    np.random.seed(seed)
    _time = np.random.choice(list(papers.keys()))
    sampn = min(len(papers[_time]), batch_size)
    pids = np.array(papers[_time])[np.random.choice(len(papers[_time]), sampn, replace = False)]
    fids = []
    edge = defaultdict(lambda: {})
    for x_id, p_id in enumerate(pids):
        f_ids = pairs[p_id]
        for f_id in f_ids:
            if f_id not in fids:
                fids += [f_id]
            edge[x_id][fids.index(f_id)] = True
    pids = np.stack([pids, np.repeat([_time], sampn)]).T
    fids = np.stack([fids, np.repeat([_time], len(fids))]).T
 
    feature, times, edge_list, _ = sample_subgraph(graph, t_range, \
                inp = {'paper': pids, 'field': fids}, sampled_depth = 3, sampled_number = 100)

    el = []
    for i in edge_list['paper']['field']['rev_PF_in_L1']:
        if i[0] in edge and i[1] in edge[i[0]]:
            continue
        el += [i]
    edge_list['paper']['field']['rev_PF_in_L1'] = el

    el = []
    for i in edge_list['field']['paper']['PF_in_L1']:
        if i[1] in edge and i[0] in edge[i[1]]:
            continue
        el += [i]
    edge_list['field']['paper']['PF_in_L1'] = el
    
    
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list, graph)
    '''
        Trace the paper_id and field_id by its own index plus the type start index
    '''
    paper_ids = np.arange(len(pids)) + node_dict['paper'][0]
    field_ids = np.arange(len(fids)) + node_dict['field'][0]
    ylabel = torch.zeros(sampn, len(cand_list))
    for x_id, p_id in enumerate(pids[:,0]):
        for f_id in pairs[p_id]:
            ylabel[x_id][cand_list.index(f_id)] = 1
    ylabel /= ylabel.sum(axis=1).view(-1, 1)
    return node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel
    
def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), train_papers, \
                                               train_pairs, train_range, batch_size))
        jobs.append(p)
    p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), valid_papers, \
                                           valid_pairs, valid_range, batch_size))
    jobs.append(p)
    return jobs

In [5]:
class HetConv(MessagePassing):
    def __init__(self, in_dim, out_dim, num_types, dropout = 0.3, **kwargs):
        super(HetConv, self).__init__(aggr='add', **kwargs)

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.num_types     = num_types
        
        self.rnns   = nn.ModuleList()
        self.aggregat_ws   = nn.ModuleList()
        for t in range(num_types):
            self.rnns.append(nn.LSTM(input_size=in_dim, hidden_size=in_dim // 2, bidirectional=True))
            self.aggregat_ws.append(nn.Linear(in_dim, out_dim))
        self.drop            = nn.Dropout(dropout)
        self.semantic_v      = nn.Linear(out_dim * 2, 1)
        
    def forward(self, node_inp, edge_index, node_type):
        return self.propagate(edge_index, node_inp=node_inp, node_type=node_type)

    def message(self, edge_index_i, node_inp_i, node_inp_j, node_type_j, num_nodes):
        '''
            i: target; j: source
        '''
        data_size   = edge_index_i.size(0)
        res_att     = torch.zeros(data_size, 1).to(node_inp_i.device)
        res_val     = torch.zeros(data_size, self.in_dim).to(node_inp_i.device)
        for _i in range(edge_index_i.max()):
            idx = (edge_index_i == _i)
            if idx.sum() == 0:
                continue
            _node_inp_i = node_inp_i[idx][0]
            _ats = []
            for nt in range(self.num_types):
                idy = idx & (node_type_j == nt)
                if idy.sum() == 0:
                    continue
                rnn = self.rnns[nt]
                _node_inp_j = rnn(node_inp_j[idy].view(1, -1, self.in_dim))[0].view(-1, self.in_dim)
                _att  = torch.cat([_node_inp_j, _node_inp_i.repeat(len(_node_inp_j), 1)], dim=-1)
                _att  = F.leaky_relu(self.semantic_v(_att), 0.2)
                res_att[idy]= _att
                res_val[idy]= _node_inp_j
        res_att = softmax(res_att, edge_index_i, data_size)
        res = res_val * res_att.view(-1, 1)
        del res_att, res_val
        return res
    
    def update(self, aggr_out):
        res = torch.zeros(aggr_out.size(0), self.out_dim).to(aggr_out.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = F.relu(aggregat_w(aggr_out[idx]))
        out = self.drop(res)
        del res
        return out

    def __repr__(self):
        return '{}(in_dim={}, out_dim={}, num_rel={})'.format(
            self.__class__.__name__, self.in_dim, self.out_dim, self.num_relations)

In [6]:
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_types, n_layers, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_types = num_types
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.aggregat_ws   = nn.ModuleList()
        self.drop          = nn.Dropout(dropout)
        for t in range(num_types):
            self.aggregat_ws.append(nn.Linear(in_dim, n_hid))
        for l in range(n_layers):
            self.gcs.append(HetConv(n_hid, n_hid, num_types))
    def forward(self, node_feature, edge_index, node_type):
        res = torch.zeros(node_feature.size(0), self.n_hid).to(node_feature.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = self.drop(res)
        del res
        for gc in self.gcs:
            meta_xs = self.drop(F.relu(gc(meta_xs, edge_index, node_type)))
        return meta_xs

In [7]:
'''
Paper-Field
'''
paper_ser = {}

train_pairs = {}
valid_pairs = {}
test_pairs  = {}

train_papers = {_time: {} for _time in train_range}
valid_papers = {_time: {} for _time in valid_range}
test_papers  = {_time: {} for _time in test_range}

for f_id in graph.edge_list['field']['paper']['PF_in_L1']:
    for p_id in graph.edge_list['field']['paper']['PF_in_L1'][f_id]:
        _time = graph.edge_list['field']['paper']['PF_in_L1'][f_id][p_id]
        if _time in train_range:
            if p_id not in train_pairs:
                train_pairs[p_id] = []
            train_pairs[p_id] += [f_id]
            train_papers[_time][p_id] = True
        elif _time in valid_range:
            if p_id not in valid_pairs:
                valid_pairs[p_id] = []
            valid_pairs[p_id] += [f_id]
            valid_papers[_time][p_id] = True
        else:
            if p_id not in test_pairs:
                test_pairs[p_id] = []
            test_pairs[p_id] += [f_id]
            test_papers[_time][p_id] = True
for _time in list(train_papers.keys()):
    if len(train_papers[_time]) < batch_size // 2:
        train_papers.pop(_time)
    else:
        train_papers[_time] = np.array(list(train_papers[_time].keys()))
for _time in list(valid_papers.keys()):
    if len(valid_papers[_time]) < batch_size // 2:
        valid_papers.pop(_time)
    else:
        valid_papers[_time] = np.array(list(valid_papers[_time].keys()))
for _time in list(test_papers.keys()):
    if len(test_papers[_time]) < batch_size // 2:
        test_papers.pop(_time)
    else:
        test_papers[_time] = np.array(list(test_papers[_time].keys()))

In [8]:
types = graph.get_types()
cand_list = [k for k in graph.edge_list['field']['paper']['PF_in_L1'] if len(graph.edge_list['field']['paper']['PF_in_L1'][k]) > 0]
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), n_layers = 3).to(device)
# gnn = torch.load('../pre-train/save/cpc_model.pt').to(device)
classifier = Classifier(256, len(cand_list)).to(device)
model = nn.Sequential(gnn, classifier)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [9]:
stats = []
pool = mp.Pool(4)
process_ids = np.arange(batch_num // 4)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(epoch_num)+1:
    '''
        Prepare Training and Validation Data
    '''
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.close()
    pool.join()
    pool = mp.Pool(4)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    model.train()
    train_losses = []
    torch.cuda.empty_cache()
    for batch in np.arange(2):
        for node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel in train_data:
            node_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device))
            res  = classifier.forward(node_rep[paper_ids])
            loss = criterion(res, ylabel.to(device))
            optimizer.zero_grad() 
            torch.cuda.empty_cache()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
            optimizer.step()
            train_losses += [loss.cpu().detach().tolist()]
            train_step += 1
            scheduler.step(train_step)
            del res, loss
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = valid_data
        node_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device))
        res  = classifier.forward(node_rep[paper_ids])
        loss = criterion(res, ylabel.to(device))
        valid_res = []

        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            valid_res += [ai[bi].tolist()]
        valid_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        if valid_ndcg > best_val:
            best_val = valid_ndcg
            torch.save(model, './save/rgt_1.pt')
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
              valid_ndcg))
        stats += [[np.average(train_losses), loss.cpu().detach().tolist()]]
        del res, loss
        if epoch % 5 == 0:
            '''
                Test
            '''
            _time = np.random.choice(list(test_papers.keys()))
            node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                           test_pairs, test_range, batch_size, test=True)
            paper_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device))[paper_ids]
            res  = classifier.forward(paper_rep)
            test_res = []
            for ai, bi in zip(ylabel, res.argsort(descending = True)):
                test_res += [ai[bi].tolist()]
            test_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in test_res])
            print(test_ndcg)
            del res
    del train_data, valid_data

Data Preparation: 191.2s


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

In [12]:
for p in gnn.parameters():
    print(p)

RuntimeError: cuda runtime error (33) : invalid resource handle at /opt/conda/conda-bld/pytorch_1565272271120/work/aten/src/THC/THCGeneral.cpp:371

In [ ]:
stats = np.array(stats)
plt.plot(stats[:,0])
plt.plot(stats[:,1])
plt.show()

In [ ]:
model.eval()
gnn, classifier = model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))

In [ ]:
best_model = torch.load('./save/rgt_1.pt')

In [ ]:
best_model.eval()
gnn, classifier = best_model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))